In [ ]:
import pandas as pd
import geopandas as gpd
from pyprojroot import here
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv(here() / 'data/base/data.csv')

In [ ]:
df = data.iloc[:,[22,74]]

In [ ]:
df.columns = ['country', 'frequency']

In [ ]:
df = df.groupby('country').median()

In [ ]:
# Load built-in world map from GeoPandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Download from Natural Earth (manual or via requests)
url = "https://naturalearth.s3.amazonaws.com/10m_cultural/ne_10m_admin_0_countries.zip"
world = gpd.read_file(url)
world = world.rename(columns={'SOVEREIGNT':'name'})

# Merge your dataframe with the GeoDataFrame
merged = world.merge(df, how='left', left_on='name', right_on='country')

In [ ]:
from shapely.geometry import box

#europe = merged[merged['continent'] == 'Europe']

# Define bounding box (lon_min, lat_min, lon_max, lat_max)
europe_box = box(-26, 34, 50, 71)  # roughly continental Europe
europe_mainland = merged.clip(europe_box)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,5))

europe_mainland.plot(column='frequency', 
            cmap='Reds', 
            legend=False, 
            ax=ax, 
            edgecolor='black',
            lw=.2,
            vmin=0,
            vmax=100,
            missing_kwds={
            'color': '.9',
            'edgecolor': 'black',
            'label': 'No data',
    })

ax.axis('off')
plt.tight_layout()

# Create the colorbar manually
sm = plt.cm.ScalarMappable(cmap='Reds')

# Add colorbar with adjustable height
cbar = fig.colorbar(sm, ax=ax, shrink=0.9, pad=0.02)

plt.savefig(here() / 'output/plots/crowding_map.png', dpi=300, bbox_inches='tight')